In [1]:
import pandas as pd
import pickle
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("data/daily_metrics_2011_2013.csv")
df = df.fillna(0).round(2)

with open('parrot.pkl', 'rb') as f:
    dates = pickle.load(f)
df["Date"] = dates
df_ = pd.read_csv('data/Fiyatveri.csv',sep=";")

In [3]:
def reverser(x):
    m = x.split(".")
    if len(m[0])>1:
        a = m[2]+"-"+m[1]+"-"+m[0]
    else:
        a = m[2]+"-"+m[1]+"-"+"0"+m[0]
    return a
def numerica(x):
    if type(x) == type(""):
        return x.replace(",", "", 1).replace(",", ".", 1)
def voll(x):
    return x.replace("-","0")

In [4]:
df

,average_clustering,density,number_of_edges,number_of_nodes,degree_assortativity_coefficient,Date
0,0.00,0.50,1,2,0.00,2009-01-09
1,0.00,0.09,10,11,-0.61,2009-01-12
2,0.00,0.03,10,20,0.00,2009-01-13
3,0.00,0.02,20,32,0.00,2009-01-14
4,0.00,0.02,21,32,-0.15,2009-01-15
...,...,...,...,...,...,...
1814,0.01,0.00,94086,62942,-0.03,2013-12-29
1815,0.01,0.00,100129,65118,-0.03,2013-12-30
1816,0.01,0.00,85026,55280,-0.03,2013-12-31
1817,0.01,0.00,68290,45349,-0.02,2014-01-01


In [5]:
df_["Date"] = df_["Date"].apply(reverser)

In [6]:
totaldf  = df_.merge(df,how="left",on="Date")

In [7]:
totaldf["Open"] =totaldf["Open"].apply(numerica)
totaldf["High"] =totaldf["High"].apply(numerica)
totaldf["Low"] = totaldf["Low"].apply(numerica) 
totaldf["Open"] = pd.to_numeric(totaldf["Open"])
totaldf["High"] = pd.to_numeric(totaldf["High"])
totaldf["Low"] = pd.to_numeric(totaldf["Low"])
totaldf["Vol"] = totaldf["Vol"].apply(voll)
totaldf["Vol"] = pd.to_numeric(totaldf["Vol"])
totaldf["Change %"] = pd.to_numeric(totaldf["Change %"])

In [8]:
#totaldf= totaldf[["Price"]]
totaldf= totaldf.drop("Date",axis=1)

In [9]:
totaldf

,Price,Open,High,Low,Vol,Change %,average_clustering,density,number_of_edges,number_of_nodes,degree_assortativity_coefficient
0,0.2,0.2,0.2,0.2,0.20,0.00,0.01,0.0,387,484,-0.32
1,0.2,0.2,0.2,0.2,5.85,0.00,0.01,0.0,297,376,-0.34
2,0.2,0.2,0.3,0.1,61.47,0.00,0.02,0.0,372,444,-0.24
3,0.2,0.2,0.2,0.2,29.70,0.00,0.01,0.0,367,473,-0.31
4,0.3,0.2,0.3,0.2,36.62,0.00,0.02,0.0,484,578,-0.30
...,...,...,...,...,...,...,...,...,...,...,...
1151,802.0,707.3,829.6,707.2,23.53,13.39,0.01,0.0,102540,67423,-0.05
1152,803.0,802.0,836.8,767.9,13.89,0.13,0.01,0.0,103492,67862,-0.04
1153,762.0,803.0,806.0,715.6,13.34,-5.11,0.01,0.0,90505,60733,-0.04
1154,785.0,762.0,804.0,735.0,7.04,3.01,0.01,0.0,94086,62942,-0.03


In [17]:
def model_creator(X_train):
    regressor = Sequential()
    regressor.add(LSTM(units = 50  , return_sequences = True , input_shape = (X_train.shape[1] ,5)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50  , return_sequences = True  ))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50    ))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50 ))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units= 1))
    regressor.compile(optimizer='adam', loss='mean_squared_error')
    return regressor

In [18]:
sc= MinMaxScaler(feature_range = (0 , 1))
totaldf_scaled =sc.fit_transform(totaldf)

X_train = []
y_train =[]
x_test = []
for i in range (60,1000):
    X_train.append(totaldf_scaled[i-60:i])
    y_train.append(totaldf_scaled[i, 0])
for i in range(1000,1156):
    x_test.append(totaldf_scaled[i-60:i])
    
X_train , y_train, x_test =np.array(X_train), np.array(y_train), np.array(x_test)

X_train =np.reshape(X_train,(X_train.shape[0] , X_train.shape[1] ,  X_train.shape[2] ))

regressor = model_creator(X_train)
#regressor.fit(X_train, y_train, epochs=100, batch_size=32)
#prediction = regressor.predict(x_test)

ValueError: Input 0 of layer "lstm_11" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 50)

In [ ]:
X_train.shape

In [13]:
regressor = model_creator(X_train)

In [14]:
regressor.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 50)            11200     
                                                                 
 dropout_4 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_5 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_6 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_6 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50)               

In [16]:
regressor.fit(X_train, y_train)

ValueError: in user code:

    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la

In [15]:
mape(prediction,totaldf_scaled[1000:,0])

NameError: name 'prediction' is not defined

In [ ]:
plt.figure(figsize=(15,6))
plt.scatter(np.arange(156),prediction, c="r",label="Prediction")
plt.scatter(np.arange(156),totaldf_scaled[1000:,0], c="g",label="Y")
plt.legend()
plt.show()

In [ ]:
prediction.shape

In [ ]:
totaldf_scaled[1000:,0].shape

In [ ]:
regressor.summary()